In [2]:
# IMPORTS

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import tensorflow as tf
import lightkurve as lk

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

import astropy
from astropy.timeseries import TimeSeries


2022-06-26 11:28:21.311697: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-26 11:28:21.311727: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [17]:
# FETCH DATA

train_df_true = pd.read_table("UID_0007562_data_AXA_001.tbl")
train_df_false = pd.read_table("UID_0007562_data_AXA_001.tbl") # USE HOYS DATA INSTEAD

print(train_df_true.iloc[:,0])


0                                   \"GLOBAL PROPERTIES"
1      \ ********************************************...
2                                    \STAR_ID = "WASP18"
3      \STAR_ID_DEFINED = "Published Star Name or ID ...
4                     \DATA_CATEGORY = "AXA Light Curve"
                             ...                        
716      2455103.6835568      0.0606581   0.06443032 ...
717      2455103.6837468      0.0608481   0.06463213 ...
718      2455103.6839268      0.0610281   0.06482334 ...
719      2455103.6841168      0.0612181   0.06502515 ...
720      2455103.6842968      0.0613981   0.06521636 ...
Name: \ ******************************************************************************, Length: 721, dtype: object


In [5]:
lc_exo = plt.plot(train_df_true[1],train_df_true[3])
lc_no_exo = plt.plot(train_df_false[1],train_df_false[3])

KeyError: 1

In [ ]:
test_df = str(input("Enter your target: "))
print()

In [ ]:
test_df1 = str(input("Enter mission: "))

In [ ]:
testsearch = lk.search_lightcurve(test_df,author=test_df1)
lc_test = testsearch.download_all()

#ls_imgs_true = []
#ls_imgs_false = []

#for x in lc_exo:
    #x.plot()
    #plt.savefig("Lightcurve_Exo_" + str(lc_exo.index(x)) + ".png")
    # export to folder

#for y in lc_no_exo:
    #plt.plot(y)
    #plt.savefig("Lightcurve_No_Exo_" + str(lc_no_exo.index(y)) + ".png")
    # export to folder

#for z in lc_test:
    #plt.plot(z)
    #plt.savefig("Lightcurve_Test_" + str(lc_test.index(z)) + ".png")
    # export to folder

lc_exo.plot()
plt.show()

In [ ]:
# CREATE TRAIN AND TEST DATASETS

train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_ds = train.flow_from_directory(
    "IMAGE FOLDER PATH",
    target_size=(150,150),
    batch_size = 32,
    class_mode = 'binary')

test_ds = test.flow_from_directory(
    "IMAGE FOLDER PATH",
    target_size=(150,150),
    batch_size = 32,
    class_mode = 'binary')

# BUILD CNN MODEL

model = keras.Sequential()

model.add(keras.layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(64,(3,3),activation="relu"))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(128,(3,3),activation="relu"))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(128,(3,3),activation="relu"))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(512,activation="relu"))

model.add(keras.layers.Dense(1,activation="sigmoid"))

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

# TRAIN DATA

model.fit_generator(
    train_ds,
    steps_per_epoch = 250,
    epochs = 10,
    validation_data = test_ds
)

# PREDICTIONS (FUNCTION)

def predictExo(filename):
    img = image.load_img(filename,target_size=(150,150))
    plt.imshow(img)

    Y = image.img_to_array(img)
    X = np.expand_dims(Y,axis=0)

    val = model.predict(X)
    print(val)

    if val == 1:
        plt.xlabel("Exoplanet detected!",fontsize=30)
    elif val == 0:
        plt.xlabel("No exoplanet detected.",fontsize=30)

predictExo(lc_test)